In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Load your data
df = pd.read_csv('cough_and_colds.csv')

# Convert 'Sold_date' to datetime format
df['Sold_date'] = pd.to_datetime(df['Sold_date'], format='%m/%d/%y')

# Set 'Sold_date' as the index
df.set_index('Sold_date', inplace=True)

# Aggregate sales data on a weekly basis for each product
weekly_sales = df.groupby([pd.Grouper(freq='W'), 
'Product_details']).agg(total_sold=('Sold_quantity', 'sum')).reset_index()

# Apply one-hot encoding to 'Product_details'
weekly_sales_encoded = pd.concat([weekly_sales, pd.get_dummies(weekly_sales['Product_details'], 
prefix='product')], axis=1).drop('Product_details', axis=1)

# Extract year and week number from 'Sold_date' for temporal features
weekly_sales_encoded['year'] = weekly_sales_encoded['Sold_date'].dt.year
weekly_sales_encoded['week_of_year'] = weekly_sales_encoded['Sold_date'].dt.isocalendar().week

# Normalize the temporal features 
scaler = MinMaxScaler()
features_columns = ['year', 'week_of_year'] + [col for col in weekly_sales_encoded.columns 
if col.startswith('product')]
weekly_sales_encoded[features_columns] = scaler.fit_transform(weekly_sales_encoded[features_columns])

# Split the data into features and target
X = weekly_sales_encoded[features_columns].values
y = weekly_sales_encoded['total_sold'].values

# Adjusting the dataset split into 70% training, 15% validation, and 15% testing
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 0.1765 of 85% is roughly 15% of the whole

# Define the model architecture
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(100, activation='relu'),  # Additional hidden layer
    Dense(1, activation='relu')    # Output layer
])


# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model with the validation data
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10)

# Evaluate the model
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'RMSE on test set: {rmse}')

# Prepare forecasting features for Feb 1 to 8, 2024, correctly this time
forecast_results = []
for product_col in [col for col in features_columns if col.startswith('product')]:
    forecast_period_features = np.zeros((1, len(features_columns)))
    # Find the index of the year, week_of_year, and current product column
    year_index = features_columns.index('year')
    week_of_year_index = features_columns.index('week_of_year')
    product_index = features_columns.index(product_col)
    
    # Normalize year and week_of_year values
    forecast_period_features[0, year_index] = (2024 - scaler.data_min_[year_index]) / (scaler.data_max_[year_index] - scaler.data_min_[year_index])
    forecast_period_features[0, week_of_year_index] = (5 - scaler.data_min_[week_of_year_index]) / (scaler.data_max_[week_of_year_index] - scaler.data_min_[week_of_year_index])
    forecast_period_features[0, product_index] = 1  # Activate current product

    # Predict and store the result
    predicted_quantity = model.predict(forecast_period_features).flatten()[0]
    product_name = product_col.replace('product_', '')  # Assuming prefix 'product_' is used
    forecast_results.append((product_name, predicted_quantity))

# Display forecasted quantities for each product in a DataFrame
forecast_df = pd.DataFrame(forecast_results, columns=['Product', 'Forecasted Quantity']).sort_values(by='Forecasted Quantity', ascending=False)
print(forecast_df)

Epoch 1/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 4668.1953 - val_loss: 634.1934
Epoch 2/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - loss: 2771.3848 - val_loss: 541.5370
Epoch 3/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 1124.0905 - val_loss: 549.6208
Epoch 4/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 1377.8650 - val_loss: 532.6157
Epoch 5/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 359us/step - loss: 1415.3828 - val_loss: 532.0411
Epoch 6/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step - loss: 1292.4480 - val_loss: 543.3792
Epoch 7/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 1517.6675 - val_loss: 552.0582
Epoch 8/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 1562.7201 - val_loss: 556.8892
Epoch 9/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 1821.2382 - val_loss: 529.6396
Epoch 10/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 1506.2766 - val_loss: 597.4480
Epoch 11/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - l

247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - loss: 646.0891 - val_loss: 701.0989
Epoch 65/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - loss: 853.4518 - val_loss: 604.9840
Epoch 66/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 764.3755 - val_loss: 576.9319
Epoch 67/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - loss: 1131.3408 - val_loss: 572.7250
Epoch 68/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 1449.5220 - val_loss: 562.0101
Epoch 69/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 669.9749 - val_loss: 548.4013
Epoch 70/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 350us/step - loss: 1084.0560 - val_loss: 616.3907
Epoch 71/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 646.2079 - val_loss: 864.8979
Epoch 72/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 801.1120 - val_loss: 579.9918
Epoch 73/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 349us/step - loss: 760.2226 - val_loss: 671.8893
Epoch 74/100
247/247 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - loss: 685.99

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
                                              Product  Forecasted Quantity
33                              Neozep Tab 100&#039;s           110.842094
53                            Symdex-D Tab 100&#039;s            80.841179
4                               Bioflu Tab 100&#039;s            57.039139
12                      Decolgen Forte Tab 100&#039;s            32.536499
55                       Tuseran forte Cap 100&#039;s            20.007126
43                             Plemex Forte 600mg Cap            14.989124
51                      Symdex-D FORTE Tab 100&#039;s            13.728006
25                  NON-Drowsy  Neozep Tab 100&#039;s            12